# Amazon Dataset Script

### Import Statements

In [28]:
import pandas as pd
import csv

### Root for csv files

In [4]:
root = "H:\\AmazonDataset\\amazon_reviews_us_"

### Create Customer Data csv File

In [48]:
headers = ['customer_id', 'num_reviews', 'num_five_star_ratings', 'num_four_star_ratings',
          'num_three_star_ratings', 'num_two_star_ratings', 'num_one_star_ratings', 'total_helpful_votes', 'vine',
          'num_verified_purchases']

with open(root+'customer_data.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(headers)
csvfile.close()

### Read Apparel_v1_00.tsv

In [42]:
fields = ['customer_id', 'review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']

tsv_data = pd.read_csv(root+"Apparel_v1_00.tsv", sep='\t', error_bad_lines=False, usecols=fields )
    
    
print(tsv_data.head())

   customer_id       review_id  star_rating  helpful_votes  total_votes vine  \
0     32158956  R1KKOXHNI8MSXU          4.0            0.0          0.0    N   
1      2714559  R26SP2OPDK4HT7          5.0            1.0          2.0    N   
2     12608825   RWQEDYAX373I1          5.0            0.0          0.0    N   
3     25482800  R231YI7R4GPF6J          5.0            0.0          0.0    N   
4      9310286  R3KO3W45DD0L1K          5.0            0.0          0.0    N   

  verified_purchase                                    review_headline  \
0                 Y  ★ THESE REALLY DO WORK GREAT WITH SOME TWEAKING ★   
1                 Y                    Favorite for winter. Very warm!   
2                 Y                         Great Socks for the money.   
3                 Y                                         Slick hat!   
4                 Y                               I would do it again!   

                                         review_body review_date  
0  Thes

In [31]:
tsv_data.query('customer_id == 33320432')

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
1045017,33320432,R4RLM7KB5II77,B00L3XT8WY,1.0,8.0,10.0,N,Y,I hate it,Ordered a large but fits like a small.,2014-09-09


In [56]:
for i in range(len(tsv_data)):
    customer_id = tsv_data.values[i][0]
    star_rating = tsv_data.values[i][2]
    helpful_votes = tsv_data.values[i][3]
    vine = tsv_data.values[i][5]
    num_reviews = 1
    
    print(star_rating)
    
    if tsv_data.values[i][6] == 'Y':
        verified_purchases = 1
    else:
        verified_purchases = 0
        
    print(verified_purchases)
    
    if star_rating == 1.0:
        num_five_star_ratings = 0
        num_four_star_ratings = 0
        num_three_star_ratings = 0
        num_two_star_ratings = 0
        num_one_star_ratings = 1
    elif star_rating == 2.0:
        num_five_star_ratings = 0
        num_four_star_ratings = 0
        num_three_star_ratings = 0
        num_two_star_ratings = 1
        num_one_star_ratings = 0
    elif star_rating == 3.0:
        num_five_star_ratings = 0
        num_four_star_ratings = 0
        num_three_star_ratings = 1
        num_two_star_ratings = 0
        num_one_star_ratings = 0
    elif star_rating == 4.0:
        num_five_star_ratings = 0
        num_four_star_ratings = 1
        num_three_star_ratings = 0
        num_two_star_ratings = 0
        num_one_star_ratings = 0
    elif star_rating == 5.0:
        num_five_star_ratings = 1
        num_four_star_ratings = 0
        num_three_star_ratings = 0
        num_two_star_ratings = 0
        num_one_star_ratings = 0
        
    row = [customer_id, num_reviews, num_five_star_ratings, num_four_star_ratings, num_three_star_ratings, num_two_star_ratings,
          num_one_star_ratings, helpful_votes, vine, verified_purchases]
    
    df = pd.read_csv(root+"customer_data.csv")
    
    if customer_id in df.values:
        df.loc[df['customer_id'] == customer_id, 'num_reviews'] += num_reviews
        print(df.loc[df['customer_id'] == customer_id, 'num_reviews'])
        
    else:
        with open(root+'customer_data.csv', 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(row)
        csvfile.close()
    
    break

4.0
1
0    2
Name: num_reviews, dtype: int64
